Import libraries

In [4]:
import pandas as pd
import numpy as np
import math

In [5]:
DATA_FOLDER = 'data'
CSV_FILE = 'food_data.csv'

In [7]:
all_datas = pd.read_csv(DATA_FOLDER + "/" + CSV_FILE, sep='\t')

/home/pirate/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,3,5,24,25,26,28,35,36,41,49,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
for col in all_datas.columns.values:
    print("- ", col)

-  code
-  url
-  creator
-  created_t
-  created_datetime
-  last_modified_t
-  last_modified_datetime
-  product_name
-  generic_name
-  quantity
-  packaging
-  packaging_tags
-  brands
-  brands_tags
-  categories
-  categories_tags
-  categories_en
-  origins
-  origins_tags
-  manufacturing_places
-  manufacturing_places_tags
-  labels
-  labels_tags
-  labels_en
-  emb_codes
-  emb_codes_tags
-  first_packaging_code_geo
-  cities
-  cities_tags
-  purchase_places
-  stores
-  countries
-  countries_tags
-  countries_en
-  ingredients_text
-  allergens
-  allergens_en
-  traces
-  traces_tags
-  traces_en
-  serving_size
-  serving_quantity
-  no_nutriments
-  additives_n
-  additives
-  additives_tags
-  additives_en
-  ingredients_from_palm_oil_n
-  ingredients_from_palm_oil
-  ingredients_from_palm_oil_tags
-  ingredients_that_may_be_from_palm_oil_n
-  ingredients_that_may_be_from_palm_oil
-  ingredients_that_may_be_from_palm_oil_tags
-  nutrition_grade_uk
-  nutrition_grade_f

We can deduce the usefull colums for our project:
product_name
sugars_100g
fat_100g
nutrition_grade_fr
countries
countries_tags
origins
origins_tags
manufacturing_places
manufacturing_places_tags
glycemic-index_100g


In [50]:
usefull_datas = all_datas[['code',
                    'product_name',
                   'countries',
                   'origins',
                   'origins_tags',
                   'manufacturing_places',
                   'first_packaging_code_geo',
                   'sugars_100g',
                   'fat_100g']]

toto = usefull_datas[['code','countries']]

In [54]:
toto = toto.groupby('countries').count()

In [55]:
toto

countries
Afrique du Sud                                                                                                                                                                                                                        1
Afrique du Sud,Swaziland                                                                                                                                                                                                              1
Albania                                                                                                                                                                                                                               1
Albania,France                                                                                                                                                                                                                        1
Albania,Italia                                                